## Transactions
#### 03.4 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-28 (started 2021-12-19)

- Load credentials
- Interact with the blockchain
- Execute a payment from Python
- Add a note to a payment

## Setup
Starting with this chapter 3.4, the lines below will always automatically load ...
* The accounts MyAlgo, Alice, Bob, Charlie, Dina
* The Purestake credentials
* The functions in `algo_util.py`

In [4]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [5]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
import algosdk.error
import json

In [6]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA
GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4
MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM
ZFTSGNBBFDH544IDUVB7S4B67CX6PJL6MMFJYT3GGDH2GWJ4N7H6VKRU2Y
OH6I6P7ZFKG6I5BBHB7R6C7BLKVCRAJUV5QTR3BZQZINPLXTWD3B4MVEKA


## Connecting to the Algorand Blockchain via API
+ We choose to connect via Purestake API
+ Requires Purestake token to authenticate
    - See 03.3_WSC_Credentials
- API **address** stored ...
    - For the testnet in `cred['algod_test']`
    - For the mainnet in `cred['algod_main']`
- API **crendentials** stored in `cred['purestake_token']`
  - Note: this is already the pair `{'X-Api-key': 'your token'}`
  - To obtain token alone use `cred['purestake_token']['X-Api-key']`
- See also: https://developer.algorand.org/tutorials/creating-python-transaction-purestake-api/  

In [ ]:
# Today we work with the testnet
algod_token   = ''                        # Only needed if we have our own server
algod_address = cred['algod_test']        # TESTNET, alternatively cred['algod_main'] 
purestake_token = cred['purestake_token'] # Authentication token pair {'X-Api-key': '(your token'}

# Initialize the algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address, headers=purestake_token)

#### Test the connection
- Our first Python access of the blockchain
- What's the last block? Verify on https://testnet.algoexplorer.io
- Note that block count on testnet is larger (Why?)

In [ ]:
algod_client.status()["last-round"]

27737829

### Check the accounts on the blockchain
- Mainnet https://algoexplorer.io 
- Testnet https://testnet.algoexplorer.io 
- Alternative indexers are https://algoscan.app and https://explorer.perawallet.app

In [ ]:
# Create a link to directly access your MyAlgo account
print('https://algoexplorer.io/address/'+MyAlgo['public'])
print('https://testnet.algoexplorer.io/address/'+MyAlgo['public'])

https://algoexplorer.io/address/33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA
https://testnet.algoexplorer.io/address/33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA


### Fund with testnet Algos
- https://bank.testnet.algorand.network/
- https://testnet.algoexplorer.io/dispenser
- https://dispenser.testnet.aws.algodev.network
- Fund `MyAlgo`, `Alice` and `Bob`. How many test ALGOs did you get?

### Obtain holdings

In [ ]:
# Get holdings of testnet Algos
address = Alice["public"]
algod_client.account_info(address)["amount"]

20381000

In [ ]:
# Holdings are in micro Algo ... convert
algo_precision = 1e6
algo_amount    = algod_client.account_info(address)["amount"]/algo_precision
print(f"Address {address} has {algo_amount} test algos")

Address GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4 has 20.381 test algos


#### Suggested parameters for a transaction (on the test network)
* Get standard parameters from API
* Simplifies creation of a transaction

In [ ]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

{
    "first": 27737834,
    "last": 27738834,
    "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "gen": "testnet-v1.0",
    "fee": 0,
    "flat_fee": false,
    "consensus_version": "https://github.com/algorandfoundation/specs/tree/44fa607d6051730f5264526bf3c108d51f0eadb6",
    "min_fee": 1000
}


**Note:** `first` and `last` are suggested params for first and last round valid. Compare to current round


## A first payment transaction

#### Step 1: prepare and create unsigned transaction using `PaymentTxn`

In [ ]:
# Parameters
sp       = algod_client.suggested_params()       # suggested params
amount   = 0.42
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Create (unsigned) TX
txn = PaymentTxn(sender = MyAlgo['public'],     # <--- From
                 sp = sp, 
                 receiver = 'WSC234WDSXHLWICSZ5DVJNRZLJHHK23D3ZSP6MS2CJNII4ZDQTC367VUN4',     # <---- To
                 amt = amt_microalgo           # <---- Amount in Micro-ALGOs
                )
print(json.dumps(vars(txn), indent=4))

{
    "sender": "GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4",
    "fee": 1000,
    "first_valid_round": 27737835,
    "last_valid_round": 27738835,
    "note": null,
    "genesis_id": "testnet-v1.0",
    "genesis_hash": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "group": null,
    "lease": null,
    "type": "pay",
    "rekey_to": null,
    "receiver": "MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM",
    "amt": 100000,
    "close_remainder_to": null
}


In [ ]:
# Interesting: we already have a txid
print(txn.get_txid())

HK4IWFR6WK5DDY5ATG3O7P7G3WQ4GINYGZX7B7UL7IZCOUOZLQQQ


In [84]:
# Is it already on the blockchain? 
# No ... not yet sent
print('https://testnet.algoexplorer.io/tx/'+txn.get_txid())

https://testnet.algoexplorer.io/tx/7POM3CAVK5LLH4ZR3GV7FDJDOVNB6QDEYEWISXPVZC2LNGQBH7NQ


#### Step 2: sign

In [38]:
stxn = txn.sign(MyAlgo['private'])                 # <---- Alice signs with private key

# The new stxn object consists of
print(stxn.transaction)                           # same
print('')
print(stxn.signature)                             # new

{'sender': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA', 'fee': 1000, 'first_valid_round': 27738031, 'last_valid_round': 27739031, 'note': b'159268198', 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'WSC234WDSXHLWICSZ5DVJNRZLJHHK23D3ZSP6MS2CJNII4ZDQTC367VUN4', 'amt': 420000, 'close_remainder_to': None}

uw47qLYkAHW4FCPqnbuB5zJB9kdINLbtkX1geaNXTsZAp7c9LP9XOMNRyssWAdC7U/p2ztq65uPM9OldCLpCBg==


In [ ]:
# The transaction ID is the same, and still nothing on the blockchain
print(stxn.get_txid())
print('https://testnet.algoexplorer.io/tx/'+stxn.get_txid())

HK4IWFR6WK5DDY5ATG3O7P7G3WQ4GINYGZX7B7UL7IZCOUOZLQQQ
https://testnet.algoexplorer.io/tx/HK4IWFR6WK5DDY5ATG3O7P7G3WQ4GINYGZX7B7UL7IZCOUOZLQQQ


#### Step 3: send

In [ ]:
txid = algod_client.send_transaction(stxn)

# The freshly submitted transaction on the blockchain
txinfo = algod_client.pending_transaction_info(txid)
print(txinfo)

{'pool-error': '', 'txn': {'sig': 'p1kjMd2H50D9bEqSm8Pm9gj1L3FpJ//24MBT2lb2ovPooenpiC0xZxeN8QDM+SZLhv5hIFcEzrnjWf3GHfXDBA==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 27737835, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 27738835, 'rcv': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM', 'snd': 'GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4', 'type': 'pay'}}}


#### Step 4: Wait for confirmation

In [ ]:
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27737840.
Waiting for round 27737840 to finish.
Waiting for round 27737841 to finish.
Transaction HK4IWFR6WK5DDY5ATG3O7P7G3WQ4GINYGZX7B7UL7IZCOUOZLQQQ confirmed in round 27737842.


In [ ]:
# Note that txinfo has now a 'confirmed-round'
print(txinfo)

{'confirmed-round': 27737842, 'pool-error': '', 'txn': {'sig': 'p1kjMd2H50D9bEqSm8Pm9gj1L3FpJ//24MBT2lb2ovPooenpiC0xZxeN8QDM+SZLhv5hIFcEzrnjWf3GHfXDBA==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 27737835, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 27738835, 'rcv': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM', 'snd': 'GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4', 'type': 'pay'}}}


In [90]:
# Now we can check the tx also on algoexplorer
print('https://testnet.algoexplorer.io/tx/'+txid)

https://testnet.algoexplorer.io/tx/7POM3CAVK5LLH4ZR3GV7FDJDOVNB6QDEYEWISXPVZC2LNGQBH7NQ


## Add a note to a transaction

In [35]:
# Step 1a: Prepare
sp       = algod_client.suggested_params()       # suggested params

amount    = 0.42
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Step 1b: The note
note_txt  = "159268198"
note_byte = note_txt.encode()

In [36]:
# Step 1c: create (unsigned) TX
txn = PaymentTxn(sender=MyAlgo['public'],
                 sp=sp, 
                 receiver = "WSC234WDSXHLWICSZ5DVJNRZLJHHK23D3ZSP6MS2CJNII4ZDQTC367VUN4",
                 amt=amt_microalgo, 
                 note=note_byte
                 )
print(txn)

{'sender': '33SG2MXXXQHP2ZMJUJ2DRKLRKTZ7DJBGKHFREPYCE3RXCUPYRL2LN57BSA', 'fee': 1000, 'first_valid_round': 27738031, 'last_valid_round': 27739031, 'note': b'159268198', 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'WSC234WDSXHLWICSZ5DVJNRZLJHHK23D3ZSP6MS2CJNII4ZDQTC367VUN4', 'amt': 420000, 'close_remainder_to': None}


In [23]:
#account1Balance = Balance(Txn.accounts[1]) # get the balance of Txn.accounts[1]

NameError: name 'Balance' is not defined

In [39]:
# Step 2+3: sign and send TX
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)
print("Send transaction with txID: {}".format(txid))

Send transaction with txID: 35HFKUFYGHRSJRPNZBASMCFCKXEDPQLSD3WUEWFRHGYBQ3EJMTGA


In [40]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27738047.
Transaction 35HFKUFYGHRSJRPNZBASMCFCKXEDPQLSD3WUEWFRHGYBQ3EJMTGA confirmed in round 27738047.


In [26]:
print(txinfo)

{'confirmed-round': 27737888, 'pool-error': '', 'txn': {'sig': 'Z3a8RfsyFPSi9nSi3HOFwVP7eCp/iLFVRMBM4aoL+x7G0hQm/0DzijKnbwHkve0vtDEq58etSSpf7jykbVg3CA==', 'txn': {'amt': 100000, 'fee': 1000, 'fv': 27737845, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 27738845, 'note': 'UGF5aW5nIGJhY2sgZm9yIGxhc3QgZGlubmVy', 'rcv': 'MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM', 'snd': 'GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4', 'type': 'pay'}}}


In [27]:
# Step 5 (check): Extract message in txinfo and convert back
import base64
note_base64 = txinfo['txn']['txn']['note']
print(note_base64)
note_byte   = base64.b64decode(note_base64)
print(note_byte)
note_txt   = note_byte.decode()
print(note_txt)

UGF5aW5nIGJhY2sgZm9yIGxhc3QgZGlubmVy
b'Paying back for last dinner'
Paying back for last dinner


In [28]:
# Check on Algoexplorer
print("https://testnet.algoexplorer.io/tx/"+txid)

https://testnet.algoexplorer.io/tx/YNEKD6IBOEA77LKCH6LB2FCOITJLCIXDFMAJOLXA23BUNQHY23PQ


## Exercises

**Exercise 1:** Send 0.82 ALGO from Dina to Alice with a thank you note

In [29]:
# Your Python code goes here
# Parameters
sp       = algod_client.suggested_params()       # suggested params
amount   = 0.82
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Create (unsigned) TX
txn = PaymentTxn(sender = Dina['public'],     # <--- From
                 sp = sp, 
                 receiver = Alice['public'],     # <---- To
                 amt = amt_microalgo           # <---- Amount in Micro-ALGOs
                )

# Step 1b: The note
note_txt  = "Paying back for last coffee"
note_byte = note_txt.encode()
print(json.dumps(vars(txn), indent=4))

{
    "sender": "OH6I6P7ZFKG6I5BBHB7R6C7BLKVCRAJUV5QTR3BZQZINPLXTWD3B4MVEKA",
    "fee": 1000,
    "first_valid_round": 27737907,
    "last_valid_round": 27738907,
    "note": null,
    "genesis_id": "testnet-v1.0",
    "genesis_hash": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "group": null,
    "lease": null,
    "type": "pay",
    "rekey_to": null,
    "receiver": "GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4",
    "amt": 820000,
    "close_remainder_to": null
}


In [30]:
# Step 1c: create (unsigned) TX
txn = PaymentTxn(sender=Dina['public'],
                 sp=sp, 
                 receiver = Alice['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )
print(txn)

{'sender': 'OH6I6P7ZFKG6I5BBHB7R6C7BLKVCRAJUV5QTR3BZQZINPLXTWD3B4MVEKA', 'fee': 1000, 'first_valid_round': 27737907, 'last_valid_round': 27738907, 'note': b'Paying back for last coffee', 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4', 'amt': 820000, 'close_remainder_to': None}


In [31]:
# Step 2+3: sign and send TX
stxn = txn.sign(Dina['private'])
txid = algod_client.send_transaction(stxn)
print("Send transaction with txID: {}".format(txid))

Send transaction with txID: DTHSDGHQR6PIAJ6WVUDNQ6RQBLZFIR6WPGUCPWPAQ3VX3XU6SYZQ


In [32]:
#Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27737910.
Waiting for round 27737910 to finish.
Transaction DTHSDGHQR6PIAJ6WVUDNQ6RQBLZFIR6WPGUCPWPAQ3VX3XU6SYZQ confirmed in round 27737911.


In [33]:
# Step 5 (check): Extract message in txinfo and convert back
import base64
note_base64 = txinfo['txn']['txn']['note']
print(note_base64)
note_byte   = base64.b64decode(note_base64)
print(note_byte)
note_txt   = note_byte.decode()
print(note_txt)

UGF5aW5nIGJhY2sgZm9yIGxhc3QgY29mZmVl
b'Paying back for last coffee'
Paying back for last coffee


In [34]:
# Check on Algoexplorer
print("https://testnet.algoexplorer.io/tx/"+txid)

https://testnet.algoexplorer.io/tx/DTHSDGHQR6PIAJ6WVUDNQ6RQBLZFIR6WPGUCPWPAQ3VX3XU6SYZQ


**Exercise 2:** Obtain the (new) ALGO holdings of Alice

In [ ]:
# Your Python code goes here

## Things that do not and will not work
Following are a few things that deliberately don't work.

In [115]:
# Need to import this to be able to read error messages
import sys, algosdk.error

### Overspending
* Alice sends more than she owns

In [116]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [117]:
# Step 3b: Send
txid = algod_client.send_transaction(signed_txn)

AlgodHTTPError: TransactionPool.Remember: transaction WE26AR3PSGXEGHHN5K7C4PJ65KWYT5JKXLIUJIS7TOLZSBVVEJ6A: overspend (account GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4, data {AccountBaseData:{Status:Offline MicroAlgos:{Raw:11437000} RewardsBase:27521 RewardedMicroAlgos:{Raw:0} AuthAddr:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ TotalAppSchema:{_struct:{} NumUint:0 NumByteSlice:0} TotalExtraAppPages:0 TotalAppParams:0 TotalAppLocalStates:0 TotalAssetParams:0 TotalAssets:0 TotalBoxes:0 TotalBoxBytes:0} VotingData:{VoteID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] SelectionID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] StateProofID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] VoteFirstValid:0 VoteLastValid:0 VoteKeyDilution:0}}, tried to spend {100000000})

#### Can we *catch the error* and get a better structured error message?
* **Note:** error occurse when sending the transaction

In [118]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [119]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                   # print entire error message
    if ("overspend" in str(err)):                # check for specific type of error
        print("Overspend error")         
    txid = None

TransactionPool.Remember: transaction 4UGAHRK4QOXWGEQEHNSFD2TLNVJKBMBOI4H6HGBEKDUJNOWKO6YQ: overspend (account GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4, data {AccountBaseData:{Status:Offline MicroAlgos:{Raw:11437000} RewardsBase:27521 RewardedMicroAlgos:{Raw:0} AuthAddr:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ TotalAppSchema:{_struct:{} NumUint:0 NumByteSlice:0} TotalExtraAppPages:0 TotalAppParams:0 TotalAppLocalStates:0 TotalAssetParams:0 TotalAssets:0 TotalBoxes:0 TotalBoxBytes:0} VotingData:{VoteID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] SelectionID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] StateProofID:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] VoteFirstValid:0 VoteLastValid:0 VoteKeyDilution:0}}, tried to spend {100000000})
Overspend error


#### What happens if we wait for the failed transaction to complete?

In [120]:
# Step 4: Wait for confirmation
try:
    txinfo = wait_for_confirmation(algod_client, txid)
    print(txinfo)
except TypeError as err:                                       # obtain error message
    # print entire error message (rather cryptic!)
    print(err)
    # Give better error message
    print("txid is empty")

Empty transaction id.
None


### Wrong signature
Bob tries to sign a transaction from Alice to Bob

In [121]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Bob['private'])                        # <----------------- wrong person signs!

In [123]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("should have been authorized" in str(err)):                # check for specific type of error
        print("Wrong signature error")         
    txid = None

TransactionPool.Remember: transaction W3ZKIWU4LOGGX2JB7N6LTDUMY6B7DZEMUXPIP435PMLNKJNUIVDQ: should have been authorized by GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4 but was actually authorized by MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM
Wrong signature error


### Sending the *indentical* transaction twice
* "Identical" means ...
    * same Sender
    * same Recipient
    * same Ammount
    * same (suggested) parameters (including first/last round)

In [124]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

In [127]:
# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
    print("Submitted with txID: {}".format(txid))
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("transaction already in ledger" in str(err)):                # check for specific type of error
        print("Identical transaction {} has been submitted twice.".format(signed_txn.get_txid()))         
    txid = None    # check for specific type of error

TransactionPool.Remember: transaction already in ledger: 6NROFEWZEFC5SV34NV7OE6XK4W3BT4AKXO6CMDKQLIEZ37W66BEA
Identical transaction 6NROFEWZEFC5SV34NV7OE6XK4W3BT4AKXO6CMDKQLIEZ37W66BEA has been submitted twice.


**REPEAT** only step 2-3 $\rightarrow$ error message<br>
**REPEAT** only step 1-3 $\rightarrow$ no error <br>

#### See how the sp change
* Re-run this after 2-3 seconds

In [131]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

{
    "first": 27729626,
    "last": 27730626,
    "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=",
    "gen": "testnet-v1.0",
    "fee": 0,
    "flat_fee": false,
    "consensus_version": "https://github.com/algorandfoundation/specs/tree/44fa607d6051730f5264526bf3c108d51f0eadb6",
    "min_fee": 1000
}


## Appendix: Useful functions
* The following functions are included in `algo_util.py`

In [132]:
def wait_for_confirmation(client, txid):
    # client = algosdk client
    # txid = transaction ID, for example from send_payment()
    # An ufficial Algorand function

    txinfo = client.pending_transaction_info(txid)       # obtain transaction information
    current_round = algod_client.status()["last-round"]        # obtain last round number
    print("Current round is  {}.".format(current_round))
    
    # Wait for confirmation
    while ( txinfo.get('confirmed-round') is None ):            # condition for waiting = 'confirmed-round' is empty
        print("Waiting for round {} to finish.".format(current_round))
        algod_client.status_after_block(current_round)             # this wait for the round to finish
        txinfo = algod_client.pending_transaction_info(txid)    # update transaction information
        current_round += 1

    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

### *Python dict as transaction note
* For attaching a Python dict as message to a note
* Example `note_dict = {'from' : 'Bob', 'to' : 'Alice', 'message' : 'Many thanks'}

In [133]:
def note_encode(note_dict):
    # note dict ... a Python dictionary
    note_json = json.dumps(note_dict)
    note_byte = note_json.encode()     
    return(note_byte)

In [134]:
def note_decode(note_64):
    # note64 =  note in base64 endocing
    # returns a Python dict
    import base64
    message_base64 = txinfo['txn']['txn']['note']
    message_byte   = base64.b64decode(message_base64)
    message_json   = message_byte.decode()
    message_dict   = json.loads( message_json )
    return(message_dict)